# Обучение модели
В этом ноутбуке выполняется обучение модели на признаках, полученных в ноутбуке `prepare_and_save_features.ipynb`.
Цель обучения - создать модель классификации, прогнозирующей, понравится ли пользователю с определенными характеристиками пост с его определенными характеристиками. 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from dotenv import dotenv_values
from sklearn.metrics import recall_score, precision_score, f1_score
from catboost import CatBoostClassifier


RND_STATE = 2024
TEST_ROWS_COUNT = 300000
SAVE_MODEL_TO_FILE = False

In [2]:
env_dict = dotenv_values("../creds/.env")
MODEL_FILE_NAME = env_dict.get("KC_STARTML_MODEL_FILE", 
                               f"catboost_model_{dt.datetime.now()}"
                               .split(".")[0]
                               .replace(" ", "_")
                               .replace(":", "_"))

## Загрузка подготовленных данных
Перед загрузкой данных с признаками надо сначала выполнить все действия в ноутбуке `prepare_and_save_features.ipynb`, предварительно поставив флаг `SAVE_DATA_TO_CSV = True`.

In [3]:
df_user_data = pd.read_csv("../data/user_features.csv")
df_user_data.head(3)

,user_id,gender,age,country_and_city,exp_group,os_iOS,source_organic
0,200,1,34,0.000869,3,0,0
1,201,0,37,0.010972,0,0,0
2,202,1,17,0.019796,4,0,0


In [4]:
df_post_data = pd.read_csv("../data/post_features.csv")
df_post_data.head(3)

,post_id,topic,tfidf_mean,tfidf_max
0,1,0.04743,0.627607,0.731115
1,2,0.04743,1.492879,-0.773927
2,3,0.04743,1.700723,-0.894502


In [5]:
df_feed_data = pd.read_csv("../data/feed_data.csv")
df_feed_data.head(3)

,timestamp,user_id,post_id,target
0,2021-10-01 06:01:40,1859,1498,1
1,2021-10-01 06:01:40,8663,3837,1
2,2021-10-01 06:01:40,15471,2810,0


## Объединение данных в один DataFrame

In [6]:
df_views_data = (df_feed_data
                .merge(df_user_data, on="user_id")
                .merge(df_post_data, on="post_id")).sort_values("timestamp").reset_index(drop=True)
del df_feed_data

In [7]:
target_counts = df_views_data.target.value_counts()
print(f"Количества значений целевой переменной:\n{target_counts}")
print(f"Баланс классов: {target_counts[1] / target_counts[0]:.4f}")
del target_counts

Количества значений целевой переменной:
0    4861763
1     498618
Name: target, dtype: int64
Баланс классов: 0.1026


In [8]:
print(df_views_data.shape)
df_views_data.head(3)

(5360381, 13)


,timestamp,user_id,post_id,target,gender,age,country_and_city,exp_group,os_iOS,source_organic,topic,tfidf_mean,tfidf_max
0,2021-10-01 06:01:40,1859,1498,1,0,19,0.039501,3,0,0,0.047443,1.653575,-1.310722
1,2021-10-01 06:01:40,66609,3270,1,1,47,0.003155,4,0,0,0.540549,-1.439735,0.462559
2,2021-10-01 06:01:40,163409,3558,0,1,23,0.025740,2,0,1,0.540549,-1.405051,-0.182656


## Создание моделей

### Деление данных на тренировочные и тестовые

In [9]:
X_train = df_views_data.iloc[:-TEST_ROWS_COUNT, :].drop(["target", "post_id", "user_id", "timestamp"], axis=1)
y_train = df_views_data.iloc[:-TEST_ROWS_COUNT, :]["target"]
X_test = df_views_data.iloc[-TEST_ROWS_COUNT:, :].drop(["target", "post_id", "user_id", "timestamp"], axis=1)
y_test = df_views_data.iloc[-TEST_ROWS_COUNT:, :]["target"]

In [10]:
def print_model_scores(model, X_train, X_test, y_train, y_test):
    """
    Печать величин Precision, Recall и F1-score переданной модели
    по переданным тренировочным и тестовым данным
    """
    pred_train = model.predict(X_train)
    print(f"Train Precision: {precision_score(y_train, pred_train):.4}")
    print(f"Train Recall: {recall_score(y_train, pred_train):.4}")
    print(f"Train F1-score: {f1_score(y_train, pred_train):.4}")
    print("")
    pred_test = model.predict(X_test)
    print(f"Precision: {precision_score(y_test, pred_test):.4}")
    print(f"Recall: {recall_score(y_test, pred_test):.4}")
    print(f"F1-score: {f1_score(y_test, pred_test):.4}")


In [11]:
def get_final_metric_value(model, test_data, k=5):
    """
    Функция измерения финальной метрики HitRate@k
    переданной модели, по переданным тестовым данным
    """
    data = test_data.copy()
    X_test_data = data.drop(["target", "post_id", "user_id", "timestamp"], axis=1)
    y_test_date = data.target

    X_test_data["pred"] = model.predict_proba(X_test_data)[:, 1]
    X_test_data["target"] = y_test_date
    X_test_data["user_id"] = data["user_id"]
    X_test_data["post_id"] = data["post_id"]

    users_hitrate = []
    
    for user in X_test_data["user_id"].unique():
        part = X_test_data[X_test_data["user_id"]==user]
        part = part.sort_values("pred", ascending=False)
        part = part.reset_index()
        user_hitrate =  part.target[:k].max()
        users_hitrate.append(user_hitrate)
        
    result = np.mean(users_hitrate)
    print(f"Среднее HitRate@{k} по пользователям из теста: {result}")
    
    return result

### Модель CatBoost

#### Обучение финальной модели

In [12]:
%%time
model = CatBoostClassifier(random_seed=RND_STATE, verbose=False, 
                           class_weights = {0: 1, 1:10},
                           iterations=1000,
                           l2_leaf_reg=1,
                           learning_rate=0.4,
                           depth=4)

model.fit(X_train, y_train)

print_model_scores(model, X_train, X_test, y_train, y_test)

Train Precision: 0.1449
Train Recall: 0.6739
Train F1-score: 0.2385

Precision: 0.1616
Recall: 0.6434
F1-score: 0.2583
CPU times: user 37min 56s, sys: 28.6 s, total: 38min 25s
Wall time: 14min 45s


#### Измерение финальной метрики

In [13]:
get_final_metric_value(model, 
                       df_views_data.iloc[-TEST_ROWS_COUNT:, :], 
                       k=5)

Среднее HitRate@5 по пользователям из теста: 0.5392049598832969


0.5392049598832969

## Сохранение модели в файл ## 

In [14]:
if SAVE_MODEL_TO_FILE:
    model.save_model(f"../models/{MODEL_FILE_NAME}", format="cbm")